In [6]:
# LangChain 필요 컴포넌트 임포트
from langchain.chat_models import ChatOpenAI  # LLM 모델 (ChatGPT)
from langchain.document_loaders import UnstructuredFileLoader  # 텍스트 파일 로더
from langchain.text_splitter import CharacterTextSplitter  # 문서 분할기
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings  # 텍스트 임베딩 도구
from langchain.vectorstores import FAISS  # 벡터 데이터베이스
from langchain.storage import LocalFileStore  # 로컬 캐시 저장소
from langchain.prompts import ChatPromptTemplate  # 프롬프트 템플릿
from langchain.schema.runnable import RunnablePassthrough  # 체인 구성 유틸리티
from langchain.memory import ConversationBufferMemory # 메모리??

# LLM 모델 초기화 (낮은 temperature로 일관된 응답 유도)
llm = ChatOpenAI(
    temperature=0.1,
)

# 대화 내용을 요약하여 저장하는 메모리 초기화
# 단순한 메모리 저장 방식
# 대화 내용을 시간 순서대로 모두 저장하는 가장 기본적인 메모리 타입입니다.
# 이전 대화들을 그대로 버퍼에 저장하여 유지합니다.
# 메모리 구조
# 대화는 input(사용자 입력)과 output(AI 응답) 쌍으로 저장됩니다.
# 모든 대화 기록이 순차적으로 저장되어 컨텍스트로 활용될 수 있습니다.
buffer_memory = ConversationBufferMemory(
    llm=llm,  # 요약에 사용할 언어 모델
    memory_key="history",  # 메모리를 참조할 때 사용할 키
    return_messages=True   # 메시지 객체 형태로 반환
)

# 임베딩 캐시 저장소 설정
cache_dir = LocalFileStore("./.cache/")

# 문서 분할 설정
# chunk_size: 각 분할 크기, chunk_overlap: 분할 간 중복되는 부분
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 텍스트 파일 로드
loader = UnstructuredFileLoader("./files/document.txt")

# 문서를 설정된 크기로 분할
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# 임베딩 결과 캐싱 설정 (API 비용 절감)
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 분할된 문서를 벡터화하여 FAISS에 저장
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 유사도 검색을 위한 리트리버 생성
# what is retriver?
# 한개의 스트링을 받아 질문이나 그와 관련성이 있는 문서를 얻기위한 쿼리 등
# 출력값은 문서의 리스트임
retriver = vectorstore.as_retriever()

# 질문-답변을 위한 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant.
            Answer questions using only the following context.
            and return the answer in Korean.
            If you don't know the answer just say you don't know,
            don't make it up:\n\n{context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 전체 체인 구성:
# 1. 질문에 관련된 문서 검색
# 2. 검색된 문서를 컨텍스트로 프롬프트 생성
# 3. LLM으로 답변 생성
# RunnablePassthrough : 입력 값을 그대로 전달하는 역할
chain = (
    # {
    #     "context": retriver,  # 문서 검색
    #     "question": RunnablePassthrough(),  # 사용자 질문 전달
    # }
    # >>> error 발생 ???
    # TypeError: expected string or buffer
    # Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
    # error 발생 이유 : ???
    # lambda 사용 이유 : 체인 구성 시 입력 값을 함수로 전달하여 동적으로 처리하기 위함
    {
        "context": lambda x: retriver.get_relevant_documents(x["question"]),  # 문서 검색
        "question": lambda x: x["question"],  # 사용자 질문 전달
    }    
    | prompt
    | llm
)

# 체인 실행 함수
def invoke_chain(question):
    # 체인 실행
    result = chain.invoke({"question": question})
    # 대화 내용을 메모리에 저장
    buffer_memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

# Victory Mansions에 대한 설명 요청
invoke_chain("Aaronson 은 유죄인가요?")

content='제가 알기로는 Aaronson이 유죄임을 알지 못합니다.'


In [7]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content="그가 쓴 메시지는 'FREEDOM IS SLAVERY'와 'TWO AND TWO MAKE FIVE'입니다."


In [8]:
invoke_chain("Julia 는 누구인가요?")

content='Julia는 Winston과 사랑을 나눈 여성입니다.'
